# Semantic Search on a Corpus

In [1]:
# Import Packages
import json
import numpy as np
import pandas as pd
import tiktoken
from joblib import load

## Import Data

In [2]:
# Import Data
filepath = "C:\\Users\\hlmq\\OneDrive - Chevron\\Desktop\\Projects\\Company Relationships\\Data\\Exxon\\"
filename = "CompanyProfileNewsData_withEmbeddings_withChunking.json"

chunked_df = pd.read_json(str(filepath)+str(filename))
len(chunked_df)

3000

In [3]:
# ================== #
# Tokenize Text
# ================== #

# TODO: change tiktoken out to real tokenizer... or use tiktoken

def get_tokens(text_2_encode: str, tokenizer=None):
    """
    Tokenize text in a string.

    Initialize a tokenizer if tokenizer == None.
    """

    if tokenizer is None:
        tokenizer = tiktoken.encoding_for_model("text-davinci-003")
    return tokenizer.encode(text=text_2_encode)


def get_num_tokens(text_2_encode: str, **kwargs):
    """
    Count the number of tokens in a string.
    """
    return len(get_tokens(text_2_encode=text_2_encode, **kwargs))


# ================== #
# Chunk Text
# ================== #

def group_words(s, n):
    """
    Subset a larger string into a list of smaller strings by number of words.
    """
    words = s.split()
    for i in range(0, len(words), n):
        yield ' '.join(words[i:i+n])


# ================== #
#  Get Embeddings
# ================== #

def get_embeddings(text=None, model=None):
    """
    Generate embeddings on a string of text.
    """
    if model==None:
        model = load('./model/SentBERTmodel.pkl')

    return model.encode(text)


# ================== #
#  Calculate Vector Similarity
# ================== #

def vector_similarity(x: "list[float]", y: "list[float]") -> float:
    """
    Returns the similarity between two vectors.

    Because embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


# ================== #
#  Order Chunks by Similarity
# ================== #

def measure_embedding_similarity(
    query: str,
    embeddings
):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embeddings(query)

    return [vector_similarity(query_embedding, embedding) for embedding in embeddings]


# ================== #
#  Get Similar Texts
# ================== #

def get_similar_texts(df, k):
    """
    Slice a dataframe on the top k results.  Sort the sliced dataframe descending on similarity score.

    If there are repeated results in top 5, keep them all.
    """
    response = df.nlargest(k, columns=['similarity score'],keep='all')
    response = response.sort_values(by='similarity score', ascending=False)
    return response

In [4]:
# ================== #
#  Run (score.py)
# ================== #

def run(question: str, k: int, embeddings, df) -> dict[str,str]:

    # Retrieve Top K Most Similar Results
    df['similarity score'] = measure_embedding_similarity(question, embeddings)

    # Count number of tokens in each article
    df['token count'] = df['n_summary'].apply(get_num_tokens)
    
    # Return Chunks With Highest Similarity (Text)
    response = get_similar_texts(df, k)

    # Remove embeddings
    keep_columns = ['n_title', 'n_link', 'n_date_published', 'n_summary', 'token count', 'similarity score']
    response = response[keep_columns]
    
    return response

In [5]:
# Generate embeddings
#embeddings = get_embeddings(chunked_df['n_summary'])
#chunked_df['embeddings'] = embeddings.tolist()

## Ask Questions

In [6]:
response = run("is ExxonMobil investing in advanced recycling of plastic", 5, chunked_df['embeddings'], chunked_df)

response.style.set_properties(subset=['n_summary'], **{'width': '900px'})

,n_title,n_link,n_date_published,n_summary,token count,similarity score
923,ExxonMobil starts operations at large-scale advanced recycling facility,https://corporate.exxonmobil.com/news/newsroom/news-releases/2022/1214_exxonmobil-starts-operations-at-large-scale-advanced-recycling-facility,2022-12-14 00:00:00,"ExxonMobil today announced successful startup of one of the largest advanced recycling facilities in North America. The facility at the company’s integrated manufacturing complex in Baytown, Texas, uses proprietary technology to break down hard-to-recycle plastics and transform them into raw materials for new products. It is capable of processing more than 80 million pounds of plastic waste per year, supporting a circular economy for post-use plastics and helping divert plastic waste currently sent to landfills. ExxonMobil starts operations at large-scale advanced recycling facility Share Print Top Baytown facility can process more than 80 million pounds per year of plastic waste Certified circular products meet demand for more sustainable materials, diverting plastic waste from landfills Planning additional global capacity to recycle 1 billion pounds annually of used plastics by year-end 2026 “We’ve proven our proprietary advanced recycling technology at Baytown, and now we’re leveraging our scale and integration to increase production of certified circular plastics to meet growing demand,” said Karen McKee, president of ExxonMobil Product Solutions Company. “There is substantial demand for recycled plastics, and advanced recycling can play an important role by breaking down plastics that could not be recycled in traditional, mechanical methods. We are collaborating with government, industry and communities to scale up the collection and sorting of plastic waste that will improve recycling rates and help our customers around the world meet their sustainability goals.” Since the start of pilot operations at Baytown last year, ExxonMobil has recycled nearly 15 million pounds of plastic waste. The",312,0.817022
1414,Plastic Producers Ramp Up Feedstock Recycling Partnerships,https://www.bnef.com/shorts/15927?e=RSS:MixedFeed,2023-03-06 10:05:00Z,"Dow, ExxonMobil among those investing in feedstock recyclingSupply agreements are most popular to source materials Feedstock recycling partnerships and investments tracked by BloombergNEFPlastics producers are accelerating investment in feedstock recycling through supply agreements and joint ventures to produce more recycled plastics. BloombergNEF tracked a record number of activities in 2022. Feedstock recycling uses a chemical process to break down plastic waste into a hydrocarbon mix that can be used to make new, virgin-grade plastic again. Many plastics producers like Dow and ExxonMobil are betting on feedstock recycling to boost their recycled production. Dow aims to produce 3 million metric tons of sustainable plastic by 2030. Supply agreements account for the largest share of feedstock recycling activities at 51%, as sourcing recycled feedstock remains a challenge for producers. The likes of TotalEnergies and CP Chem have formed agreements with recyclers including Nexus Circular, New Hope Energy and Plastic Energy. As demand for recycled plastics grows, more chemicals companies may choose to vertically integrate through partnerships and investments to source the feedstock needed to meet production targets.This Short is from: 1Q 2023 Sustainable Materials Market Outlook: Policy Push",232,0.785153
6,ExxonMobil tests advanced recycling of plastic waste at Baytown facilities,https://corporate.exxonmobil.com/news/newsroom/news-releases/2021/0225_exxonmobil-tests-advanced-recycling-of-plastic-waste-at-baytown-facilities,2021-02-25 00:00:00,"ExxonMobil has completed the initial phase of a plant trial of a proprietary advanced recycling process for converting plastic waste into raw materials for production of high-value polymers. The trial, at the company’s existing facilities in Bay

In [7]:
response = run("What are Exxon's decarbonization goals?", 5, chunked_df['embeddings'], chunked_df)

response.style.set_properties(subset=['n_summary'], **{'width': '900px'})

,n_title,n_link,n_date_published,n_summary,token count,similarity score
588,ExxonMobil announces ambition for net zero greenhouse gas emissions by 2050,https://corporate.exxonmobil.com/news/newsroom/news-releases/2022/0118_exxonmobil-announces-ambition-for-net-zero-greenhouse-gas-emissions-by-2050,2022-01-18 00:00:00,"ExxonMobil today announced its ambition to achieve net zero greenhouse gas emissions for operated assets by 2050, backed by a comprehensive approach to develop detailed emission-reduction roadmaps for major facilities and assets. ExxonMobil announces ambition for net zero greenhouse gas emissions by 2050 Share Print Top Comprehensive approach centered on detailed Scope 1 and Scope 2 emission-reduction roadmaps for major operated assets Ambition supported by 2030 emission-reduction plans, including net-zero plans for Permian Basin operations Company strategy tested for resiliency against a range of net-zero scenarios, including IPCC and IEA The net-zero ambition is contained in the company’s Advancing Climate Solutions - 2022 Progress Report, formerly known as the Energy & Carbon Summary. The net-zero aspiration applies to Scope 1 and Scope 2 greenhouse gas emissions and builds on ExxonMobil’s 2030 emission-reduction plans, which include net-zero emissions for Permian Basin operations and ongoing investments in lower-emission solutions in which it has extensive experience, including carbon capture and storage, hydrogen and biofuels. “ExxonMobil is committed to playing a leading role in the energy transition, and Advancing Climate Solutions articulates our deliberate approach to helping society reach a lower-emissions future,” said Darren Woods, chairman and chief executive officer. “We are developing comprehensive roadmaps to reduce greenhouse gas emissions from our operated assets around the world, and where we are not the operator, we are working with our partners to achieve similar emission-reduction results.” The report provides details of how ExxonMobil’s business strategy is resilient when tested against a range of Paris-aligned net-zero",342,0.579303
591,ExxonMobil announces ambition for net zero greenhouse gas emissions by 2050,https://corporate.exxonmobil.com/news/newsroom/news-releases/2022/0118_exxonmobil-announces-ambition-for-net-zero-greenhouse-gas-emissions-by-2050,2022-01-18 00:00:00,"for the entire corporation. Similarly, absolute flaring and methane emissions are expected to decrease by 60% and 70%, respectively by 2030. ExxonMobil has regularly updated emission-reduction plans as technologies and policies have evolved and will continue to do so. When final data is collected and analyzed, the company expects to report it achieved its 2025 emission-reduction plans as of year-end 2021, including a 15-20% reduction in greenhouse gas intensity for its upstream operations, compared to 2016 levels. ExxonMobil’s strategy is outlined in Advancing Climate Solutions and leverages its advantages in scale, integration, technology and people to build globally competitive businesses that lead industry in earnings and cash flow growth across a broad range of future scenarios.",148,0.577810
2190,Our position on climate policy and carbon pricing,https://corporate.exxonmobil.com/news/newsroom/news-releases/statements/our-position-on-climate-policy-and-carbon-pricing,2021-07-02 00:00:00,"Their comments are entirely inconsistent with our commitment to the environment, transparency and what our employees and management team have worked toward since I became CEO four years ago. I want to make our position clear. We understand the tremendous challenge represented by climate change and have fully supported the Paris Agreement since its inception. Our scientists are working to develop innovative solutions to help reduce emissions, with a focus on the highest emitting and most difficult to decarbonize sectors of the economy: commercial transportation, power generation and heavy industry. It is in these areas where we beli

In [8]:
response = run("How much is Exxon planning to invest in Carbon Capture, Utilization, and Storage?", 5, chunked_df['embeddings'], chunked_df)

response.style.set_properties(subset=['n_summary'], **{'width': '900px'})

,n_title,n_link,n_date_published,n_summary,token count,similarity score
2777,ExxonMobil outlines plans to grow long-term shareholder value in lower carbon future,https://corporate.exxonmobil.com/news/newsroom/news-releases/2021/0303_exxonmobil-outlines-plans-to-grow-long-term-shareholder-value-in-lower-carbon-future,2021-03-03 00:00:00,"of investment opportunities while reducing capital spending by more than 30 percent in 2020 as a result of the pandemic. The company also reduced cash operating expenses by 15 percent in 2020 and expects permanent structural savings of $6 billion a year by the end of 2023 versus 2019. Future spending plans take into account potential market volatility as the economy recovers from the pandemic. “Our investments are expected to generate returns of greater than 30 percent,” said Woods. “And 90 percent of our upstream investments in resource additions, including in Guyana, Brazil and the U.S. Permian Basin, generate a 10 percent return at $35 per barrel or less. Downstream investments improve net cash margin by 30 percent and our Chemical investments grow high-value performance products by 60 percent.” To grow shareholder value through the transition to a lower carbon economy, ExxonMobil has focused its extensive research and development portfolio on technologies to address hard to de-carbonize sectors of the economy responsible for approximately 80 percent of energy-related emissions -- commercial transportation, power generation and heavy industry. The company’s newly created business, ExxonMobil Low Carbon Solutions, was established to commercialize low-emission technologies, and will initially focus on carbon capture and storage (CCS), the process of capturing CO2 that would otherwise be released into the atmosphere from industrial activity, and injecting it into deep geologic formations for safe, secure and permanent storage. ExxonMobil is the industry leader in CCS technology and has more than 30 years of experience capturing carbon. The",313,0.749658
1462,"ExxonMobil planning hydrogen production, carbon capture and storage at Baytown complex",https://corporate.exxonmobil.com/news/newsroom/news-releases/2022/0301_exxonmobil-planning-hydrogen-production-carbon-capture-and-storage-at-baytown-complex,2022-03-01 00:00:00,"emissions from its operated assets by 2050. It also would enable the site to manufacture lower-emissions products for its customers. Access to surplus hydrogen and CO2 storage capacity would be made available to nearby industry. The project would form ExxonMobil’s initial contribution to a broad, cross-industry effort to establish a Houston carbon capture and storage hub with an initial target of about 50 million metric tons of CO2 per year by 2030, and 100 million metric tons by 2040. Evaluation and planning for the Baytown project are ongoing and, subject to stakeholder support, regulatory permitting and market conditions, a final investment decision is expected in two to three years. ExxonMobil has extensive experience with hydrogen and already produces about 1.5 billion cubic feet per day. The company is uniquely positioned to participate in the growing hydrogen market and is evaluating strategic investments to increase the use of this important lower-emissions energy technology. Equally important is the company’s more than 30 years of experience capturing and permanently storing CO2. ExxonMobil has cumulatively captured more human-made CO2 than any other company and has an equity share of about one-fifth of the world’s carbon capture and storage capacity, which amounts to about 9 million metric tons per year. ExxonMobil Low Carbon Solutions business was established to commercialize low-emission technologies and is focusing on carbon capture and storage, hydrogen and biofuels – technologies where the company can leverage its core competencies and competitive advantages. Over the next six years, the company plans to invest",314,0.742003
2357,"ExxonMobil presentation details s

In [10]:
response = run("How much is Exxon planning to invest in Hydrogen?", 5, chunked_df['embeddings'], chunked_df)

response.style.set_properties(subset=['n_summary'], **{'width': '900px'})

,n_title,n_link,n_date_published,n_summary,token count,similarity score
2972,"Investors Can’t Ignore US Climate Act, Hydrogen Evangelist Says",https://www.bnef.com/news/1122901?e=RSS:MixedFeed,2023-03-07 03:33:10Z,"$2 billion, there it’s going to cost $1 billion because they’re going to fund half of it,” he said. “And then they’re going to make sure that they cut the first $3 of cost of producing green hydrogen.” By Rob Verdonck",65,0.708357
434,US Hubs Will Soon Produce World’s Cheapest Green Hydrogen,https://www.bnef.com/insights/30841?e=RSS:MixedFeed,2023-02-24 14:11:56Z,The world’s cheapest green hydrogen will soon flow from projects that are being set up across the US. The $8 billion committed by the Biden administration for developing regional clean hydrogen clusters – and the $3 per kilogram that will be available as a tax credit thanks to the Inflation Reduction Act – will ensure that.,65,0.683064
2949,BP Economist Warns Against Too Much Focus on Decarbonization,https://www.bnef.com/news/1125315?e=RSS:MixedFeed,2023-03-14 18:43:50Z,"February, the European producer reduced its goal to slash oil and gas output by 2030 to between 20% and 30%, from between 35% and 40% previously. Forget Peak Oil Demand: A Thirst for Barrels Puts $100 in View Oil consumption is heading for a record this year, according to the International Energy Agency, which advises major economies. Supply — buffeted by Russia’s invasion of Ukraine, a slowdown in US shale growth and lackluster investment in production — hasn’t kept pace. BP’s estimates for oil investment requirements are lower than that of OPEC, with the Organization of Petroleum Exporting Countries expecting a need for $12.1 trillion to 2045. The European oil major anticipates that, at most, the sector will need slightly more than $400 billion a year until 2050. OPEC, the IEA and BP’s estimates “vary widely,” Cohen said. “Its not sensible to get bogged down in the details, what’s clear is we need to keep investing as we have been in the past.” The company is also targeting investment in green hydrogen and biofuels. It is focused on growing these areas in Latin America, said Angelica Ruiz, BP’s senior vice president for Latin America. “We are not only doing more, but we have acquired new companies that underpin this new growth engine that we believe is critical.” ( Updates with information on BP and oil market dynamics in fourth, fifth and sixth paragraphs. Adds tout. ) By Amy Stillman",321,0.656689
2794,ExxonMobil statement regarding President Biden Letter to Oil Industry,https://corporate.exxonmobil.com/news/newsroom/news-releases/2022/0615_exxonmobil-statement-regarding-president-biden-letter-to-oil-industry,2022-06-15 00:00:00,"ExxonMobil today released the following statement in response to a letter from President Biden. ExxonMobil statement regarding President Biden Letter to Oil Industry Share Print Top We have been in regular contact with the administration to update the President and his staff on how ExxonMobil has been investing more than any other company to develop U.S. oil and gas supplies. This includes investments in the U.S. of more than $50 billion over the past five years, resulting in an almost 50% increase in our U.S. production of oil during this period. Globally, we’ve invested double what we’ve earned over the past five years -- $118 billion on new oil and gas supplies compared to net income of $55 billion. This is a reflection of the company’s long-term growth strategy, and our commitment to continuously invest to meet society’s demand for our products. Specific to refining capacity in the U.S., we’ve been investing through the downturn to increase refining capacity to process U.S. light crude by about 250,000 barrels per day – the equivalent of adding a new medium-sized refinery. We kept investing even during the pandemic, when we lost more than $20 billion and had to borrow more than $30 billion to maintain investment to increase capacity to be ready for post-pan

## Output Data (if needed)

### Produce Data

In [ ]:
"""
# Chunk Articles
len_words = 250

titles=[]
links=[]
dates=[]
text=[]
chunk_number=[]

for i in range(0,len(df)):

    # Determine whether to chunk an article based on length
    if (len(df['n_summary'][i]) <= len_words):
        # TODO: Keep the row for all the records with fewer words than len_words
        titles.append(df['n_title'][i])
        links.append(df['n_link'][i])
        dates.append(df['n_date_published'][i])
        text.append(df['n_summary'][i])
        chunk_number.append(i)

    else:
        # Chunk article
        article = df['n_summary'][i]
        chunked_article = list(group_words(article,len_words))

        # Keep track of attributes
        article_title = df['n_title'][i]
        article_link = df['n_link'][i]
        article_date = df['n_date_published'][i]

        for i in range(0, len(chunked_article)):
            titles.append(article_title)
            links.append(article_link)
            dates.append(article_date)
            text.append(chunked_article[i])
            chunk_number.append(i)


# Build dataframe from lists

chunked_df = pd.DataFrame()
chunked_df['n_title'] = titles
chunked_df['n_link'] = links
chunked_df['n_date_published'] = dates
chunked_df['n_summary'] = text
chunked_df['chunk_number'] = chunk_number
"""

In [ ]:
"""

OUTPUT_filepath = "C:\\Users\\hlmq\\OneDrive - Chevron\\Desktop\\Projects\\Company Relationships\\Data\\Exxon\\"
OUTPUT_file = "CompanyProfileNewsData_withEmbeddings_withChunking.json"

chunked_df.to_json(str(OUTPUT_filepath)+str(OUTPUT_file), orient='records')
"""